In [1]:
import torch
import torch.nn as nn
from diffusers import AutoencoderKL,StableDiffusionPipeline, DPMSolverMultistepScheduler


/home/master/10/jimmyliu/miniconda3/envs/ldm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/master/10/jimmyliu/miniconda3/envs/ldm/lib/python3.8/site-packages/onnxruntime/capi/onnxruntime_validation.py:114: UserWarning: WARNING: failed to get cudart_version from onnxruntime build info.
  warnings.warn("WARNING: failed to get cudart_version from onnxruntime build info.")


In [ ]:
def compare_model(model1,model2):
    for p1, p2 in zip(model1.parameters(), model2.parameters()):
        if p1.data.ne(p2.data).sum() > 0:
            return False
    return True

In [ ]:
model_ori=torch.load('/tmp3/jimmyliu/HiDDeN_experiment/wm_decoder_checkpoint/whiten45.pt').to('cpu')
model_com=torch.load('/tmp3/jimmyliu/HiDDeN_experiment/wm_decoder_checkpoint/compare.pt').to('cpu')

In [2]:
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float32)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)

#vae_ori=AutoencoderKL.from_pretrained(model_id, subfolder="vae")
#vae_com=AutoencoderKL.from_pretrained('/tmp3/jimmyliu/HiDDeN_experiment/finetune_SD/runs/test 2023.05.28--14-20-05/checkpoints/vae_1')
vae_ori=pipe.vae

Fetching 15 files: 100%|██████████| 15/15 [00:00<00:00, 117817.53it/s]
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [3]:
t=torch.rand(1,3,512,512)
print(vae_ori(t).sample)

False


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 2>:2                                                                              │
│                                                                                                  │
│   1 t=torch.rand(1,3,512,512)                                                                    │
│ ❱ 2 print(vae_ori(t).sample)                                                                     │
│   3                                                                                              │
│                                                                                                  │
│ /home/master/10/jimmyliu/miniconda3/envs/ldm/lib/python3.8/site-packages/torch/nn/modules/module │
│ .py:1110 in _call_impl                                                                           │
│                                                                                                  │
│   1107 │   │   # this function, and just call forward.                                           │
│   1108 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_hooks o  │
│   1109 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1110 │   │   │   return forward_call(*input, **kwargs)                                         │
│   1111 │   │   # Do not call functions when jit is used                                          │
│   1112 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1113 │   │   if self._backward_hooks or _global_backward_hooks:                                │
│                                                                                                  │
│ /home/master/10/jimmyliu/miniconda3/envs/ldm/lib/python3.8/site-packages/diffusers/models/autoen │
│ coder_kl.py:169 in forward                                                                       │
│                                                                                                  │
│   166 │   │   x = sample                                                                         │
│   167 │   │   posterior = self.encode(x).latent_dist                                             │
│   168 │   │   print(sample_posterior)                                                            │
│ ❱ 169 │   │   raise NotImplementedError                                                          │
│   170 │   │   if sample_posterior:                                                               │
│   171 │   │   │   z = posterior.sample(generator=generator)                                      │
│   172 │   │   else:                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NotImplementedError

In [ ]:
ans=compare_model(vae_ori.encoder,vae_com.encoder)
print(ans)
ans=compare_model(vae_ori.quant_conv,vae_com.quant_conv)
print(ans)
ans=compare_model(vae_ori.decoder,vae_com.decoder)
print(ans)
ans=compare_model(vae_ori.post_quant_conv,vae_com.post_quant_conv)
print(ans)